# Filtering a signal

In this notebook we are going to look at how interpolation and decimation works.

## Interpolation
The signal we are going to interpolate is going to be a sampled version of  
$$
\begin{align*}
    x(t)&=\sin(2\pi f_1 t + \phi) 
\end{align*}
$$
where $f_1$ is the frequency. 

First we do the imports

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
plt.rcParams['figure.figsize'] = (6.0, 3)


We now implement the signal and sample it.
The parameters we are using are:
$$
\begin{align*}
    f_1&=50~\text{Hz}\\
    f_s&=800~\text{Hz}\\
    N &= 256
\end{align*}
$$

In [ ]:
N = 256 
fs = 800 # Hz
f1 = 50 # Hz
phi = np.pi/5.3

ts = 1/fs # s
n = np.arange(0,N,1)

x = np.sin(2*np.pi*f1*n*ts+phi) 

plt.figure(1)
plt.clf()
plt.plot(n,x,'--.')
plt.xlabel('$n$')
plt.ylabel('$x[n]$')
plt.xlim([0,100]) # Only show the first 100 samples
plt.grid()
plt.tight_layout()
plt.show()

## Interpolating with a factor of 4

We are now going to interpolate the signal with a factor of $L=4$
The implementation her will be the direct one, though we could use the *scipy.signal* method *upfirdn*  https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.upfirdn.html#scipy.signal.upfirdn 

or the *scipy.signal.resample* method

### Insert the zeros and plot the fft

In [ ]:
L = 4
NL = N*L
# Make the new array
xl = np.zeros(NL)
xl[0::L]=x
fsl = fs * L # The new fs 

plt.figure(11)
plt.clf()
# Plot the frequency response of the filter
plt.plot(np.arange(NL)*fsl/N, np.abs(np.fft.fft(xl)),'--.')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('FFT of xl')
plt.grid()
plt.tight_layout()
plt.show()

### Make the lowpass filter

In [ ]:
att =60 #dB, the dampening in the stopband
cutoff =fs/2-50  # The cutoff frequency in Hz
stop = fs/2+50# Desired stopband start frequency, 

Nt =  att/(22*(stop/fsl-cutoff/fsl))  # The number of taps 
Nt = int(np.ceil(Nt)) # Round up number of taps (coefficients)
if Nt%2==0: # Vi må ha et odde antall
    Nt = Nt+1
h = signal.remez(Nt, [0, cutoff, stop, 0.5*fsl], [1, 0], fs=fsl)
plt.figure(2)
plt.clf()
plt.plot(h,'.')
plt.show()
display(cutoff)
display(Nt)

We now plot the frequency response of the filter

In [ ]:
plt.figure(3)
plt.clf()
# Plot the frequency response of the filter
w, H = signal.freqz(h, worN=NL)
plt.plot(0.5*fsl*w/np.pi, 20*np.log10(np.abs(H)), linewidth=2)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain (dB)')
plt.title('Frequency Response')
plt.grid()
plt.tight_layout()
plt.show()

## Using the filter 


In [ ]:
y = signal.convolve(xl, h, mode='valid') # Convolving the filter coefficients and the signal

# Amplify it
ya = y*L

# Make the time-axes 
# For y we need to shift the output samples such that they come at the expected time. 
#  This is done by shifting with the delay G = D*ts/2
tsy = ts/L
ty = (np.arange(len(ya))*ts + (Nt-1)*ts/2) / L

tx = n*ts
# Plotting the result
plt.figure(4)
plt.clf()
plt.plot(ty, ya,'--.',color='blue')
plt.plot(tx,x,'--.',color='red')
plt.xlabel('Tid (s)')
plt.ylabel('Amplitude')
plt.legend(['Interpolated', 'Original'])
plt.tight_layout()
plt.show()


# Plot the DFT
plt.figure(5)
plt.clf()
Y = np.fft.fft(ya)
plt.plot(np.arange(len(ya))*fsl/len(ya),np.abs(Y),'-o')
plt.xlabel('$f$ (Hz)')
plt.ylabel('Magnitude')
plt.xlim([0,fsl/2]) # Only showing half the plot
plt.grid()
plt.tight_layout()
plt.show()


## Decimation 

First we set the factor and make the filter

In [ ]:
M = 4 
NM = N//M
fsm = fs/M
att = 60 #dB, the dampening in the stopband
cutoff =fsm/2-50  # The cutoff frequency in Hz
stop = fsm/2+50# Desired stopband start frequency, 
Ntm =  att/(22*(stop/fs-cutoff/fs))  # The number of taps 
Ntm = int(np.ceil(Ntm)) # Round up number of taps (coefficients)
if Ntm%2==0: # Vi må ha et odde antall
    Ntm = Ntm+1
display('Number of filter coefficients:', Ntm)
h = signal.remez(Ntm, [0, cutoff, stop, 0.5*fs], [1, 0], fs=fs)

We now plot the frequency response of the filter

In [ ]:
plt.figure(6)
plt.clf()
# Plot the frequency response of the filter
w, H = signal.freqz(h, worN=N)
plt.plot(0.5*fs*w/np.pi, 20*np.log10(np.abs(H)), linewidth=2)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain (dB)')
plt.title('Frequency Response')
plt.grid()
plt.tight_layout()
plt.show()

We now apply the filter and remove the samples  

In [ ]:
ytmp = signal.convolve(x, h, mode='valid') # Convolving the filter coefficients and the signal

yM = ytmp[0::M] # Taking out every Mt sample

Plotting the result

In [ ]:
# Make the time-axes 
# For y we need to shift the output samples such that they come at the expected time. 
# This is done by shifting with the delay G = D*ts/2
tym = np.arange(len(yM))*ts*M + (Ntm-1)*ts/(2)

tx = n*ts
# Plotting the result
plt.figure(7)
plt.clf()
plt.plot(tym, yM,'--.',color='blue')
plt.plot(tx, x,'--.',color='red')
plt.xlabel('Tid (s)')
plt.ylabel('Amplitude')
plt.legend(['Decimated', 'Original'])
plt.tight_layout()
plt.show()


# Plot the DFT
plt.figure(8)
plt.clf()
Y = np.fft.fft(yM, NM)
plt.plot(np.arange(0,NM)*fsm/NM, np.abs(Y),'-o')
plt.xlabel('$f$ (Hz)')
plt.ylabel('Magnitude')
plt.xlim([0,fsm/2]) # Only showing half the plot
plt.grid()
plt.tight_layout()
plt.show()
